### Connect to Gremlin Server

In [1]:
import os
import sys
from pathlib import Path

container_src_path = Path('/app/src/')
local_src_path = Path(Path.cwd(), 'src/')

# see if this src path exists.
# if it does, we are in a container.
# if not, we are in local.
if not container_src_path.exists():
    src_path = local_src_path
else:
    src_path = container_src_path

src_path_str = str(src_path)
if src_path_str not in sys.path:
    sys.path.insert(0, src_path_str)
    
from graph.base import g

from dotenv import load_dotenv

load_dotenv()

from concurrent.futures import ThreadPoolExecutor
from contextlib import contextmanager
from queue import Queue

# Queue to hold statements
statement_queue = Queue()

# Function to add statement to queue
def add_to_queue(statement):
    statement_queue.put(statement)

@contextmanager
def execute_gremlin_statements():
    try:
        yield add_to_queue
    finally:
        statements = []
        while not statement_queue.empty():
            statements.append(statement_queue.get())
        
        with ThreadPoolExecutor() as executor:
            futures = [executor.submit(statement) for statement in statements]
            results = [future.result() for future in futures]

        for result in results:
            print(result)


### Populate Database

In [2]:
from models.base import SessionLocal
from blockchain_data_provider import PersistentBlockchainAPIData
from graph_populate import PopulateOutputProportionGraph

delete_db = True

provider = PersistentBlockchainAPIData()
populator = PopulateOutputProportionGraph(provider)

with execute_gremlin_statements() as add_statement, \
    SessionLocal() as session:
    if delete_db:
        add_statement(lambda: populator.clear_graph())
    add_statement(lambda: populator.populate(session, range(0, 1000),
                                             show_progressbar=True))

100%|██████████| 1000/1000 [00:32<00:00, 30.83it/s]

None
None


### Get Networkx Graph

In [6]:
from graph_analyze import GraphAnalyzer

analyzer = GraphAnalyzer()



with execute_gremlin_statements() as add_statement:
    graph = analyzer.to_networkx()

TypeError: to_networkx() missing 1 required positional argument: 'subgraph_traversal'

### Find Coin Sources

In [ ]:
def get_sources(g, node_name):
    source_record = {}
    def traverse_sources(vertex, fraction=1.0):
        # Traverse incoming edges
        in_edges = g.V(vertex).inE('sent').toList()
        for edge in in_edges:
            sender = edge.outV().next()
            amount_from_sender = edge['value']
            if 'marked' in sender.properties:
                # Add record if sender is marked
                if sender['name'] in source_record:
                    source_record[sender['name']] += amount_from_sender * fraction
                else:
                    source_record[sender['name']] = amount_from_sender * fraction
            # Recursive case
            if g.V(sender).inE('sent').hasNext():
                sender_total_received = sum(e['value'] for e in g.V(sender).inE('sent').toList())
                amount_fraction = (amount_from_sender / sender_total_received) * fraction
                traverse_sources(sender, amount_fraction)

    # Start traversal
    traverse_sources(g.V().has('name', node_name).next())
    return source_record

print("Sources for Bob:")
print(get_sources(g, 'Bob'))

### Execute Some Queries

In [ ]:
from concurrent.futures import ThreadPoolExecutor

# def add_vertex():
#     return g.addV('Address').property('addr', 'some_address').next()

# with ThreadPoolExecutor() as executor:
#     future = executor.submit(add_vertex)
#     result = future.result()
    # print(result)
    
with execute_gremlin_statements() as add_statement:
    add_statement(lambda: g.addV('Address').property('addr', 'some_address').next())

v[4288]
